In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

## Carga AGEBs 1990

In [ ]:
# 1990
path_1990  = "../../Sample_Data/Raw/hermosillo/AGEBS_1990/Hermosillo_AGEBs_Poblacion_1990.shp"
census_1990 = gpd.read_file(path_1990)
agebs_1990 = census_1990[["AGEB", "P01", "geometry"]].copy()
agebs_1990['AGEB'] = agebs_1990['AGEB'].str.replace('-', '')
agebs_1990 = agebs_1990.rename(columns={'P01': 'pobtot', 'AGEB':'CVEGEO'})
agebs_1990.head()

In [ ]:
fig, ax = plt.subplots()
agebs_1990.plot(ax=ax)
ax.axis('off')
plt.show()

In [ ]:
agebs_1990.crs

## Malla Geoestadística

In [ ]:
path_data = "../../Sample_Data/Raw/"
malla_path = path_data + "malla_geoestadistica/conjunto_de_datos/nivel8.shp"
malla_geo  = gpd.read_file(malla_path)

In [ ]:
malla_geo.head()

## Reproyección de CRS

In [ ]:
# Lambert Azimuthal Equal Area (EPSG: 3035)
agebs_1990 = agebs_1990.to_crs(agebs_90.crs)
malla_geo = malla_geo.to_crs(agebs_90.crs)

## Reproyecciones

In [ ]:
import geopandas as gpd

# Realizar la intersección espacial entre las geometrías
intersection = gpd.sjoin(malla_geo, agebs_1990, how="inner", predicate='intersects')


# Calcular la proporción de área compartida
intersection['area_intersection'] = intersection.geometry.area
intersection['prop_area'] = intersection['area_intersection'] / intersection.geometry.area

# Proyectar la población
intersection['poblacion_proyectada'] = intersection['prop_area'] * intersection['pobtot']

# Agrupar por celda de la malla y sumar la población proyectada
poblacion_por_malla = intersection.groupby('codigo')['poblacion_proyectada'].sum().reset_index()

# Realizar la fusión entre poblacion_por_malla y el GeoDataFrame de la intersección
poblacion_por_malla_con_geometria = poblacion_por_malla.merge(intersection[['codigo', 'geometry']], on='codigo', how='left')
poblacion_por_malla_con_geometria = gpd.GeoDataFrame(poblacion_por_malla_con_geometria, geometry='geometry')

# Graficar la geometría con la población utilizando una escala de color
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
poblacion_por_malla_con_geometria.plot(column='poblacion_proyectada', edgecolor='black', cmap='YlGnBu', 
                                       legend=True, ax=ax, legend_kwds={'label': "Población Proyectada"})

# Superponer las geometrías de las AGEBs encima del gráfico existente
agebs_1990.plot(ax=ax, alpha=0.5, edgecolor='white')

plt.title('Población Proyectada por Celda de Malla con AGEBs')
plt.show()

In [ ]:
# Graficar la geometría con la población utilizando una escala de color
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
poblacion_por_malla_con_geometria.plot(column='poblacion_proyectada', edgecolor='black', cmap='YlGnBu', 
                                       legend=True, ax=ax, legend_kwds={'label': "Población Proyectada"})

plt.title('Población Proyectada por Celda de Malla con AGEBs')
plt.show()

In [ ]:
# Graficar la geometría con la población utilizando una escala de color
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
poblacion_por_malla_con_geometria.plot(column='poblacion_proyectada', edgecolor='black', cmap='YlGnBu', 
                                       legend=True, ax=ax, legend_kwds={'label': "Población Proyectada"})

# Superponer las geometrías de las AGEBs encima del gráfico existente
agebs_1990.plot(ax=ax, alpha=0.5, edgecolor='white')

plt.title('Población Proyectada por Celda de Malla con AGEBs')
plt.show()

In [ ]:
# Graficar la geometría con la población utilizando una escala de color
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
agebs_1990.plot(column='pobtot', edgecolor='black', cmap='YlGnBu', 
                                       legend=True, ax=ax, legend_kwds={'label': "Población Proyectada"})

plt.title('Población por AGEBs')
plt.show()